In [ ]:
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting u

In [ ]:
!pip uninstall wandb

In [ ]:
import torch
from huggingface_hub import login
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

In [ ]:
login()

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# Hugging Face Hub 설치
!pip install huggingface_hub

# Hugging Face CLI로 로그인
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install bitsandbytes-cpu

model_name = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map='cpu',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

ERROR: Could not find a version that satisfies the requirement bitsandbytes-cpu (from versions: none)
ERROR: No matching distribution found for bitsandbytes-cpu


CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
def question_gemma(question, model=model, tokenizer=tokenizer, temperature=0.0, return_answer=False):
    question = f"너는 한국은행 직원이야. 고객의 질문에 답해야 해: {question}"
    input_ids = tokenizer(question, return_tensors="pt").to("cuda")
    if temperature > 0:
        do_sample=True
    else:
        do_sample=False
    outputs = model.generate(**input_ids,
                             max_new_tokens=100,
                             do_sample=do_sample,
                             temperature=temperature)
    result = str(tokenizer.decode(outputs[0])).replace("<bos>", "").replace("<eos>", "").strip()
    if return_answer:
        return result
    else:
        print(result)

In [ ]:
question_gemma("지역화폐가 뭐야?")

In [ ]:
import pandas as pd
from datasets import Dataset

file_path = "qa_data.csv"
data = pd.read_csv(file_path)

In [ ]:
# 질문과 답변에서 불필요한 부분 제거
data['질문'] = data['질문'].str.replace('질문\n', '').str.strip()
data['답변'] = data['답변'].str.replace('답변\n', '').str.strip()

# 데이터셋 형식으로 변환
dataset = Dataset.from_pandas(data[['질문', '답변']])

# 데이터셋 확인
print(dataset)

In [ ]:
dataset['질문']

In [ ]:
dataset['답변']

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",]
)
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir='gemma2_temp',
    per_device_train_batch_size=1,
    optim="paged_adamw_32bit",
    gradient_accumulation_steps= 8,
    num_train_epochs=5,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=5e-5,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to= []
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 1024,
    dataset_text_field="답변",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

for step, batch in enumerate(trainer.get_train_dataloader()):
    inputs = batch['input_ids']
    labels = batch['labels']
    print("Input Tokens: ", tokenizer.decode(inputs[0], skip_special_tokens=True))
    print("Label Tokens: ", tokenizer.decode(labels[0], skip_special_tokens=True))
    break

In [ ]:
model.config.use_cache = False

trainer.train()

In [ ]:
model.save_pretrained("/results/gemma2_lora_model")
tokenizer.save_pretrained("/results/gemma2_lora_tokenizer")

In [ ]:
def question_gemma(question, model=model, tokenizer=tokenizer, temperature=0.0, return_answer=False):
    # 입력 질문 포맷팅
    question = f"{question}"

    # 질문을 토큰화하여 모델 입력 준비
    input_ids = tokenizer(question, return_tensors="pt").to("cuda")

    do_sample = temperature > 0

    # 모델에서 답변 생성
    outputs = model.generate(
        **input_ids,
        max_new_tokens=300,           # 최대 생성 토큰 수 제한
        do_sample=do_sample,          # 샘플링 여부 결정
        temperature=temperature,      # temperature 값 조정
        no_repeat_ngram_size=2,       # 반복되는 n-gram 방지
        early_stopping=True           # 답변이 자연스럽게 종료되면 중단
    )

    # 생성된 답변을 디코딩하여 문자열로 변환
    result = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # 반환할지 출력할지 선택
    if return_answer:
        return result
    else:
        print(result)

# 질문 테스트
question_gemma("지역화폐의 의미?", model=model, tokenizer=tokenizer)

In [ ]:
dataset['답변'][0]

In [ ]:
question_gemma("무역금융의 융자 방법에 대해 알려줘", model=model, tokenizer=tokenizer)

In [ ]:
new_file_path = "rectified_text_2023_2.txt"
with open(new_file_path, 'r', encoding='utf-8') as file:
    raw_text = file.read()

# rectified_text_2023_2.txt 파일을 QA 형식으로 변환
data_lines = raw_text.split("출력:")  # Assume '출력:' separates sections for easier processing
qa_pairs = []

for segment in data_lines:
    if "입력:" in segment:
        question = segment.split("입력:")[-1].strip()
        answer = segment.split("입력:")[0].strip()
        qa_pairs.append({'질문': question, '답변': answer})

# 새로운 데이터셋 생성
new_qa_data = pd.DataFrame(qa_pairs)
new_dataset = Dataset.from_pandas(new_qa_data[['질문', '답변']])

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/results/gemma2_lora_model")
tokenizer = AutoTokenizer.from_pretrained("/results/gemma2_lora_tokenizer")

# 모델에 동일한 PEFT (LoRA) 설정 적용
model = get_peft_model(model, peft_config)

In [ ]:
new_training_arguments = TrainingArguments(
    output_dir='gemma2_continued_temp',
    per_device_train_batch_size=1,
    optim="paged_adamw_32bit",
    gradient_accumulation_steps=8,
    num_train_epochs=3,  # 이어서 학습이므로 epoch를 줄일 수 있음
    logging_steps=10,
    warmup_steps=10,
    learning_rate=1e-5,  # 이어서 학습할 때는 학습률을 낮추는 것이 좋음
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to=[]
)

# 새로운 SFTTrainer 설정
new_trainer = SFTTrainer(
    model=model,
    train_dataset=new_dataset,
    peft_config=peft_config,
    max_seq_length=1024,
    dataset_text_field="답변",
    tokenizer=tokenizer,
    args=new_training_arguments,
    packing=False,
)

In [ ]:
new_trainer.train()

In [ ]:
model.save_pretrained("/results/gemma2_continued_lora_model")
tokenizer.save_pretrained("/results/gemma2_continued_lora_tokenizer")

In [ ]:
def question_gemma(question, model=model, tokenizer=tokenizer, temperature=0.7, max_length=256):
    input_prompt = f"입력: {question}\n출력: "
    input_ids = tokenizer.encode(input_prompt, return_tensors='pt').to("cuda")

    outputs = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("출력:")[-1].strip()

In [ ]:
print(question_gemma("2023년 기준금리 변화에 대해 설명해줘", model=model, tokenizer=tokenizer))